In [11]:
# trazendo a base que foi respondido de ura
import pymysql
import pandas as pd 
import numpy as np
import glob  
import warnings
warnings.filterwarnings('ignore')

# Estabelece a conexão com o banco de dados
connection = pymysql.connect(
    host='10.0.6.2',
    user='planejamento',
    password='@Planejamento!123',
    database='atn'
)

# Define a consulta SQL
query = """
SELECT
DATE_FORMAT(a.instante, '%d-%m') AS DATA
,a.Chave1 AS DEVEDOR
,a.Chave2 AS NOME
,a.Chave3 AS CPF
,CONCAT(a.DDD, a.Fone) AS telefone
,a.id_arquivo
,d.ARC_DESC
FROM totalinfo_2023_10 a 
	LEFT JOIN tabulacaooper top ON a.callid = top.callid
	LEFT JOIN arquivo_mailing d ON a.id_arquivo = d.ID_ARQUIVOMAILING
WHERE a.GrupoPrincipal IN(6189,6187,6191)
AND a.Agente<>''
AND top.codtabulacao <> 50
AND a.instante>='2023-10-09 08:00:00'
AND a.instante<='2023-10-09 23:00:00'
"""

respondidos_claro_net = pd.read_sql(query, connection)

respondidos_claro_net['ARC_DESC'] = respondidos_claro_net['ARC_DESC'].str.upper()
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('NET'), 'CARTEIRA'] = 'NET'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('FIXO'), 'CARTEIRA'] = 'FIXO'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('MOVEL'), 'CARTEIRA'] = 'MOVEL'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('TV'), 'CARTEIRA'] = 'TV'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('GEVENUE'), 'CARTEIRA'] = 'GEVENUE'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('.DAT'), 'CARTEIRA'] = 'GEVENUE'

respondidos_gevenue = respondidos_claro_net.query("CARTEIRA == 'GEVENUE'")


respondidos_claro_net = respondidos_claro_net.loc[:,['CPF','NOME','telefone','CARTEIRA']]

respondidos_movel = respondidos_claro_net.query("CARTEIRA == 'MOVEL'")
respondidos_tv = respondidos_claro_net.query("CARTEIRA == 'TV'")
respondidos_fixo = respondidos_claro_net.query("CARTEIRA == 'FIXO'")
respondidos_net = respondidos_claro_net.query("CARTEIRA == 'NET'")


#DANDO ALEATORIO NOS ARQUIVOS
respondidos_movel = respondidos_movel.sample(frac = 1)
respondidos_tv = respondidos_tv.sample(frac = 1)
respondidos_fixo = respondidos_fixo.sample(frac = 1)


respondidos_gevenue = respondidos_gevenue.loc[:,['DEVEDOR','NOME','telefone','CARTEIRA']]
respondidos_gevenue = respondidos_gevenue.sample(frac = 1)


In [12]:
### baixando os dfs
respondidos_movel.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Outubro\10\Markin 7\respondido cpc/respondidos_cpc_movel_1010.csv",index=False,sep=';')
respondidos_tv.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Outubro\10\Markin 7\respondido cpc/respondidos_cpc_tv_1010.csv",index=False,sep=';')
respondidos_fixo.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Outubro\10\Markin 7\respondido cpc/respondido_cpc_fixo_1010.csv",index=False,sep=';')
respondidos_net.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Outubro\10\Markin 7\respondido cpc/respondidos_cpc_net_1010.csv",index=False,sep=';')
respondidos_gevenue.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Outubro\10\Markin 7\respondido cpc/respondidos_cpc_gevenue_1010.csv",index=False,sep=';')

In [ ]:
# CARREGANDO A BASE CLARODA INTERSIC PRA SABER O ATRASO E O SALDO

########################################################## BASE INTERSIC CLARO GABRIELE ################################################################
colunas_necessarias = ["CPF","ATR_CLAR","SALDO_ABERTO_CLARO"]

caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES CLARO"
arquivos = glob.glob(caminho + "/*.txt")
#Lista de larguras do layout NET COB
lista_de_larguras = [50,50,20,50,20,50,50,100,20,50,9,9,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,17,17,19,18,8,11]
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_fwf(item, encoding='iso-8859-1', chunksize=800000, sep=";",widths = lista_de_larguras, dtype='str',usecols=colunas_necessarias)
        for df in chunks:
            dfList.append(df)
        
df_mailing_claro = pd.concat(dfList, sort=False, ignore_index=False)
pd.set_option("display.max_columns", None)


# TRATANDO CPF E CNPJ, INCLUINDO ZEROS A ESQUERDA.
df_cpfs_correto = df_mailing_claro[df_mailing_claro['CPF'].str.len()==11]
df_cnpjs_correto = df_mailing_claro[df_mailing_claro['CPF'].str.len()==14]
df_cpfs_corrigir = df_mailing_claro[df_mailing_claro['CPF'].str.len()<11]
df_cpfs_corrigir['CPF'] = df_cpfs_corrigir['CPF'].apply(lambda x: '{0:0>11}'.format(x))
df_cnpj_corrigir = df_mailing_claro[df_mailing_claro['CPF'].str.len()>11]
df_cnpj_corrigir['CPF']  = df_cnpj_corrigir['CPF'].apply(lambda x: '{0:0>14}'.format(x))
df_mailing_claro = pd.concat([df_cpfs_correto,df_cpfs_correto,df_cpfs_corrigir,df_cnpj_corrigir],ignore_index=True)\
.drop_duplicates(['CPF','ATR_CLAR','SALDO_ABERTO_CLARO'])

In [ ]:
# Trazendo os dados do respondido de ura para o arquivo da intersic com saldo e atraso
respondidos_movel = pd.merge(df_mailing_claro, respondidos_movel, right_on=["CPF"], left_on=["CPF"], how="inner")\
.loc[:,['CPF',	'SALDO_ABERTO_CLARO',	'ATR_CLAR',	'DATA',	'DEVEDOR',	'NOME',	'telefone','CARTEIRA']]\
.drop_duplicates(['CPF','DEVEDOR','NOME','telefone'])

# Trazendo os dados do respondido de ura para o arquivo da intersic com saldo e atraso
respondidos_tv = pd.merge(df_mailing_claro, respondidos_tv, right_on=["CPF"], left_on=["CPF"], how="inner")\
.loc[:,['CPF',	'SALDO_ABERTO_CLARO',	'ATR_CLAR',	'DATA',	'DEVEDOR',	'NOME',	'telefone','CARTEIRA']]\
.drop_duplicates(['CPF','DEVEDOR','NOME','telefone'])

# Trazendo os dados do respondido de ura para o arquivo da intersic com saldo e atraso
respondidos_fixo = pd.merge(df_mailing_claro, respondidos_fixo, right_on=["CPF"], left_on=["CPF"], how="inner")\
.loc[:,['CPF',	'SALDO_ABERTO_CLARO',	'ATR_CLAR',	'DATA',	'DEVEDOR',	'NOME',	'telefone','CARTEIRA']]\
.drop_duplicates(['CPF','DEVEDOR','NOME','telefone'])

#DANDO ALEATORIO NOS ARQUIVOS
respondidos_movel = respondidos_movel.sample(frac = 1)
respondidos_tv = respondidos_tv.sample(frac = 1)
respondidos_fixo = respondidos_fixo.sample(frac = 1)

respondidos_movel.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Junho\02\Markin 7\respondido cpc/respondidos_movel_cpc_0206.csv",index=False,sep=';')
respondidos_tv.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Junho\02\Markin 7\respondido cpc/respondidos_tv_cpc_0206.csv",index=False,sep=';')
respondidos_fixo.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Junho\02\Markin 7\respondido cpc/respondidos_fixo_cpc_0206.csv",index=False,sep=';')



In [ ]:
# CARREGANDO A BASE  NET DA INTERSIC PRA SABER O ATRASO E O SALDO
########################################################## BASE INTERSIC NET GABRIELE ################################################################

colunas_necessarias = ["CPF","ATR_NET","SALDO_ABERTO_NET"]

caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES NET"
arquivos = glob.glob(caminho + "/*.txt")
#Lista de larguras do layout NET COB
lista_de_larguras = [50,50,20,50,20,50,50,100,20,50,9,9,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,17,17,19,18,8,11]
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_fwf(item, encoding='iso-8859-1', chunksize=800000, sep=";",widths = lista_de_larguras, dtype='str',usecols=colunas_necessarias)
        for df in chunks:
            dfList.append(df)
        
df_mailing_net = pd.concat(dfList, sort=False, ignore_index=False)


# Trazendo os dados do respondido de ura para o arquivo da intersic com saldo e atraso
df_mailing_net = pd.merge(df_mailing_net, respondidos_net, right_on=["CPF"], left_on=["CPF"], how="inner")\
.loc[:,['CPF',	'SALDO_ABERTO_NET',	'ATR_NET',	'DATA',	'DEVEDOR',	'NOME',	'telefone','CARTEIRA']]\
.drop_duplicates(['CPF','DEVEDOR','NOME','telefone'])

df_mailing_net = df_mailing_net.sample(frac = 1)


df_mailing_net.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Junho\02\Markin 7\respondido cpc/respondidos_net_cpc_0206.csv",index=False,sep=';')



In [ ]:
# CARREGANDO A BASE  NET DA INTERSIC PRA SABER O ATRASO E O SALDO
########################################################## BASE GEVENUE ################################################################
colunas_necessarias_gevenue = ["CPF/CNPJ","AGING","VLR_TOTAL"]

caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES GEVENUE"
arquivos = glob.glob(caminho + "/*.dat")

header_gevenue = ["SENHA","AGENCIA","NOME","CPF/CNPJ","TIPO_PESSOA","SCORE_DEVEDOR","DATA_AGENDA","HORA_AGENDA","VLR_NET_TV_VIRTUA","VLR_NET_FONE","VLR_CLR_MOVEL","VLR_CLR_TV",
                  "VLR_CLR_FIXO","VLR_TOTAL","AGING","EMAIL","CONTRATO",
                  "TIPO_TELEFONE","DDI","DDD","TELEFONE","RAMAL",
                  "TIPO_TELEFONE.1","DDI.1","DDD.1","TELEFONE.1","RAMAL.1",
                      	"TIPO_TELEFONE.2","DDI.2","DDD.2","TELEFONE.2","RAMAL.2",
                        "TIPO_TELEFONE.3","DDI.3","DDD.3","TELEFONE.3","RAMAL.3",
                        "TIPO_TELEFONE.4","DDI.4","DDD.4","TELEFONE.4","RAMAL.4",
                        "TIPO_TELEFONE.5","DDI.5","DDD.5","TELEFONE.5","RAMAL.5",
                        "TIPO_TELEFONE.6","DDI.6","DDD.6","TELEFONE.6","RAMAL.6",
                        "TIPO_TELEFONE.7","DDI.7","DDD.7","TELEFONE.7","RAMAL.7"]

                        

#Lista de larguras do layout NET COB
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_csv(item, encoding='iso-8859-1', chunksize=500000, sep=";", dtype='str',names=header_gevenue)
        for df in chunks:
            dfList.append(df)
df_mailing_gevenue = pd.concat(dfList, sort=False, ignore_index=False)
df_mailing_gevenue = df_mailing_gevenue.loc[:,colunas_necessarias_gevenue]

df_mailing_gevenue

# TRATANDO CPF E CNPJ, INCLUINDO ZEROS A ESQUERDA.
df_cpfs_correto = df_mailing_gevenue[df_mailing_gevenue['CPF/CNPJ'].str.len()==11]
df_cnpjs_correto = df_mailing_gevenue[df_mailing_gevenue['CPF/CNPJ'].str.len()==14]
df_cpfs_corrigir = df_mailing_gevenue[df_mailing_gevenue['CPF/CNPJ'].str.len()<11]
df_cpfs_corrigir['CPF/CNPJ'] = df_cpfs_corrigir['CPF/CNPJ'].apply(lambda x: '{0:0>11}'.format(x))
df_cnpj_corrigir = df_mailing_gevenue[df_mailing_gevenue['CPF/CNPJ'].str.len()>11]
df_cnpj_corrigir['CPF/CNPJ']  = df_cnpj_corrigir['CPF/CNPJ'].apply(lambda x: '{0:0>14}'.format(x))
df_mailing_gevenue = pd.concat([df_cpfs_correto,df_cpfs_correto,df_cpfs_corrigir,df_cnpj_corrigir],ignore_index=True)\
.drop_duplicates(['CPF/CNPJ','AGING','VLR_TOTAL'])



respondidos_gevenue = pd.merge(df_mailing_gevenue, respondidos_gevenue, right_on=["CPF/CNPJ"], left_on=["CPF/CNPJ"], how="inner")\
.loc[:,['CPF/CNPJ',	'VLR_TOTAL',	'AGING',	'DATA',	'DEVEDOR',	'NOME',	'telefone','CARTEIRA']]\
.drop_duplicates(['CPF/CNPJ','DEVEDOR','NOME','telefone'])

respondidos_gevenue = respondidos_gevenue.sample(frac = 1)


respondidos_gevenue.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Junho\02\Markin 7\respondido cpc/respondidos_gevenue_cpc_0206.csv",index=False,sep=';')